# 识别VPN流量 🕵️‍♂️
* 使用 Tensorflow 搭建神经网络完成VPN流量的识别
* FIAT: 向前发送两个数据包之间的时间（平均值，最大值，最小值，标准差）
* BIAT: 向后发送两个数据包之间的时间（平均值，最大值，最小值，标准差）
* FLOWIAT: 形成数据流的两个数据包之间的时间（平均值，最大值，最小值，标准差）
* ACTIVE: 时间量，在变成空闲之前的活跃时间
* IDLE: 时间量，在变成活跃之前的空闲时间

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

## 1. 加载数据

In [ ]:
pd_data = pd.read_csv('../data/15s.csv')
pd_data.head()

In [ ]:
pd_data.keys()

In [ ]:
pd_data.shape

### 1.1 观察数据分布

In [ ]:
plt.figure(figsize=(20,20))
for i in range(len(pd_data.keys())-1):
    key = pd_data.keys()[i]
    plt.subplot(6,4,i+1)
    plt.xlabel(key)
    plt.hist(pd_data[key],bins=20)
plt.show()

### 1.2 处理数据
* 数据打乱
* 制作标签
* 数据归一
* 数据集划分

In [ ]:
def create_labels(labels):
    return np.where(labels=='VPN',1,0)

def create_datas(pd_data):
    shuffled_datas = pd_data.sample(frac=1.0)
    labels = create_labels(np.array(shuffled_datas.label))
    norm_datas = shuffled_datas.drop('label', axis=1)
    norm_datas_min = norm_datas.min(0)
    norm_datas_max = norm_datas.max(0)
    norm_datas = (norm_datas - norm_datas_min) / (norm_datas_max - norm_datas_min)
    numpy_datas = np.array(norm_datas)
    test_x, test_y = numpy_datas[0:2000], labels[0:2000]
    train_x, train_y = numpy_datas[2000:], labels[2000:]
    print("测试数据与标签：{} | {}".format(test_x.shape, test_y.shape))
    print("训练数据与标签：{} | {}".format(train_x.shape, train_y.shape))
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = create_datas(pd_data)

## 2. 使用 Tensorflow 建立模型

In [ ]:
learning_rate = 0.0007
epochs = 2
batch_size = 64

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

### 2.1 开始训练

In [ ]:
log = model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

### 2.2 训练过程可视化

In [ ]:
log.history

In [ ]:
plt_x = log.epoch[::10]
plt_y1 = log.history['loss'][::10]
plt_y2 = log.history['val_loss'][::10]
plt.plot(plt_x, plt_y1, 'b-')
plt.plot(plt_x, plt_y2, 'r-')
plt.show()

In [ ]:
plt_y3 = log.history['acc'][::10]
plt_y4 = log.history['val_acc'][::10]
plt.plot(plt_x, plt_y3, 'b-')
plt.plot(plt_x, plt_y4, 'r-')

### 2.3 测试数据集验证结果

In [ ]:
model.evaluate(test_x, test_y, verbose=2)